In [1]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
from delta import *
from create_spark import CreateSparkSession

In [ ]:
spark_obj = CreateSparkSession()
spark = spark_obj.session

In [7]:
spark.sql("optimize 's3://wonseokchoi-data-lake-project/lake/cassandra_replication/price/all_stocks/'")

23/09/24 05:08:53 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,totalClusterParallelism:bigint,totalScheduledTasks:bigint,autoCompactParallelismStats:struct<maxClusterActiveParallelism:bigint,minClusterActiveParallelism:bigint,maxSessionActiveParallelism:bigint,minSessionActiveParallelism:bigint>,de

In [8]:
df = spark.read \
    .format("delta") \
    .load("s3://wonseokchoi-data-lake-project/lake/cassandra_replication/price/all_stocks/")
df.printSchema()

root
 |-- time_bucket: long (nullable = true)
 |-- symbol: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- close: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- open: double (nullable = true)
 |-- transactions: long (nullable = true)
 |-- updated_at: timestamp (nullable = true)
 |-- volume: long (nullable = true)
 |-- volume_weighted: long (nullable = true)



In [9]:
df1 = df.withColumn("timestamp", F.date_format(F.from_unixtime(F.col("timestamp")/1000), "yyyy-MM-dd HH:mm:ss"))
df_cleaned = df1.drop("updated_at", "time_bucket")
df_googl = df_cleaned.filter(F.col("symbol") == "GOOGL").orderBy(F.col("timestamp").asc())
df_googl.printSchema()
df_googl.show(10)

root
 |-- symbol: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- close: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- open: double (nullable = true)
 |-- transactions: long (nullable = true)
 |-- volume: long (nullable = true)
 |-- volume_weighted: long (nullable = true)



+------+-------------------+-------+-------+-------+-------+------------+------+---------------+
|symbol|          timestamp|  close|   high|    low|   open|transactions|volume|volume_weighted|
+------+-------------------+-------+-------+-------+-------+------------+------+---------------+
| GOOGL|2018-09-06 20:00:00|   60.0|   60.0|   60.0|   60.0|           7|  4620|             60|
| GOOGL|2018-09-06 20:53:00|  60.01|  60.01|  60.01|  60.01|           1|  6000|             60|
| GOOGL|2018-09-06 20:54:00| 60.015| 60.015|  60.01|  60.01|           8|  9940|             60|
| GOOGL|2018-09-06 21:00:00|59.9995|59.9995|59.9995|59.9995|          11|  4980|             60|
| GOOGL|2018-09-06 21:03:00| 59.969| 59.969| 59.969| 59.969|           4|  6760|             59|
| GOOGL|2018-09-06 21:15:00|60.0575|60.0575|60.0575|60.0575|           1|  2000|             60|
| GOOGL|2018-09-06 21:22:00|60.0575|60.0575| 60.052|60.0575|          16| 14140|             60|
| GOOGL|2018-09-06 21:31:00|60

In [13]:
from delta.tables import DeltaTable

df_googl.coalesce(1).write.format("delta").mode("overwrite").save("s3://wonseokchoi-data-lake-project/lake/cleaned/price_cleaned/GOOGL/")

deltaTable = DeltaTable.forPath(spark, "s3://wonseokchoi-data-lake-project/lake/cleaned/price_cleaned/GOOGL/")
deltaTable.optimize()

In [10]:
DF = df_googl.toPandas()
DF.head(10)
# DF.to_csv("../data/csv/googl.csv", index=False)

,symbol,timestamp,close,high,low,open,transactions,volume,volume_weighted
0,GOOGL,2018-09-06 20:00:00,60.0000,60.0000,60.0000,60.0000,7.0,4620.0,60.0
1,GOOGL,2018-09-06 20:53:00,60.0100,60.0100,60.0100,60.0100,1.0,6000.0,60.0
2,GOOGL,2018-09-06 20:54:00,60.0150,60.0150,60.0100,60.0100,8.0,9940.0,60.0
3,GOOGL,2018-09-06 21:00:00,59.9995,59.9995,59.9995,59.9995,11.0,4980.0,60.0
4,GOOGL,2018-09-06 21:03:00,59.9690,59.9690,59.9690,59.9690,4.0,6760.0,59.0
